In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}


params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' 
}


In [2]:
#loop through pages for n pages
n = 10

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))


  0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
results

,title,link,author
0,[HTML][HTML] Consumers in a circular economy: ...,https://www.sciencedirect.com/science/article/...,"D Nainggolan, AB Pedersen, S Smed, KH Zemo… - ..."
1,[HTML][HTML] Circular economy and household e-...,https://www.sciencedirect.com/science/article/...,"D Sengupta, I Ilankoon, KD Kang, MN Chong - Mi..."
2,[HTML][HTML] Household organic waste: Integrat...,https://www.sciencedirect.com/science/article/...,"É Celestino, A Carvalho, JM Palma-Oliveira - J..."
3,"[HTML][HTML] Waste prevention, energy recovery...",https://www.sciencedirect.com/science/article/...,"I de Sadeleer, H Brattebø, P Callewaert - Reso..."
4,[HTML][HTML] Potential for circular economy in...,https://www.sciencedirect.com/science/article/...,"K Parajuly, H Wenzel - Journal of Cleaner Prod..."
...,...,...,...
5,[PDF][PDF] Effective solid waste management in...,http://www.procedia-esem.eu/pdf/issues/2017/no...,"O Oyelola, I Ajiboshin, J Okewole - … Science,..."
6,[HTML][HTML] Co-production in solid waste mana...,https://link.springer.com/article/10.1007/s113...,"OB Ezeudu, TC Oraelosi, JC Agunwamba… - … Scie..."
7,[HTML][HTML] Waste treatment company decision-...,https://www.sciencedirect.com/science/article/...,"S Snellinx, J Van Meensel, S Farahbakhsh… - Jo..."
8,[HTML][HTML] Food waste biorefinery towards ci...,https://www.sciencedirect.com/science/article/...,"S Talekar, K Ekanayake, B Holland, C Barrow - ..."
